# Video Load

In [1]:
import cv2
import torch
video_path = '/DATA_17/ij/peopleNet_test/0001_compressed.mp4'
def proc(video_path):
    frame_list = []
    cap = cv2.VideoCapture(video_path)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)  # float
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float
    fps = cap.get(cv2.CAP_PROP_FPS)
    cnt = 0
    if cap.isOpened() :    
        while True:
            cnt+=1
            ret, frame = cap.read()
            if ret == False :
                break
            if cnt < 0:
                continue
            
            infe_frame = torch.from_numpy(frame).to(torch.device("cuda"),non_blocking=True)
            infe_frame = infe_frame.permute(2, 0, 1) 
            input_data = dict()
            input_data["framedata"] = {"frame":infe_frame}
            input_data["framedata"]['meta'] = {'source' : {'channel_id' : str(1_10), 'frame_count' : cnt }}
            input_data["bbox"] = [0,0,infe_frame.shape[2],infe_frame.shape[1]]
            input_data["scenario"] = "s"   
            input_data["data"] = None   
            frame_list.append(input_data)
    return frame_list

frame_list = proc(video_path)
print('frame load done')

frame load done


# Video Load with Batch

In [10]:
import cv2
import torch
batch_size = 16
video_path = '/DATA_17/ij/peopleNet_test/0001_compressed.mp4'
def proc(video_path, batch_size):
    frame_list = []
    batch_stack = []
    cap = cv2.VideoCapture(video_path)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)  # float
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float
    fps = cap.get(cv2.CAP_PROP_FPS)
    cnt = 0
    if cap.isOpened() :    
        while True:
            cnt+=1
            ret, frame = cap.read()
            if ret == False :
                if batch_stack:
                    frame_list.append(batch_stack)
                break
            if cnt < 0:
                continue
            
            infe_frame = torch.from_numpy(frame).to(torch.device("cuda"),non_blocking=True)
            infe_frame = infe_frame.permute(2, 0, 1) 
            input_data = dict()
            input_data["framedata"] = {"frame":infe_frame}
            input_data["framedata"]['meta'] = {'source' : {'channel_id' : str(1_10), 'frame_count' : cnt }}
            input_data["bbox"] = [0,0,infe_frame.shape[2],infe_frame.shape[1]]
            input_data["scenario"] = "s"   
            input_data["data"] = None
            batch_stack.append(input_data)
            
            if cnt % batch_size == 0:
                frame_list.append(batch_stack)
                batch_stack = []
                
    return frame_list

frame_list = proc(video_path, batch_size)
print('frame load done, frame_list size : ', len(frame_list), '*', batch_size)

frame load done, frame_list size :  75 * 16


# PeopleNet Load

In [3]:
import cv2
import torch
# from interface_dynamic import PEOPLE_DETECTOR
from interface_people_hj import PEOPLE_DETECTOR
import time
torch.cuda.init()


                
weights = '/DATA_17/ij/peopleNet_test/best_model_people.trt'

pe = PEOPLE_DETECTOR()
pe.load(weights)

def run(frame_list):
    for i in range(1) : 
        start = time.time()
        for input_data in frame_list : 
            x, scale_list = pe.run_inference([input_data],[])

        print(f'end : {time.time() -start}')
run(frame_list)
raise

## 1 [234, 432, 464, 948]
## 1 [1530, 347, 1690, 742]
## 1 [1504, 248, 1602, 637]
## 2 [234, 432, 464, 948]
## 2 [1518, 349, 1684, 742]
## 3 [234, 432, 464, 948]
## 3 [1502, 349, 1650, 740]
## 4 [232, 430, 464, 948]
## 4 [1478, 351, 1642, 740]
## 5 [234, 430, 464, 948]
## 5 [1474, 345, 1638, 740]
## 6 [232, 430, 464, 948]
## 6 [1472, 345, 1634, 740]
## 7 [232, 430, 464, 948]
## 7 [1470, 337, 1632, 738]
## 8 [232, 430, 464, 948]
## 8 [1470, 337, 1632, 738]
## 9 [232, 430, 464, 948]
## 9 [1470, 335, 1628, 738]
## 10 [232, 430, 464, 948]
## 10 [1470, 337, 1622, 738]
## 11 [232, 430, 464, 948]
## 11 [1470, 331, 1628, 736]
## 12 [232, 430, 464, 948]
## 12 [1470, 329, 1622, 732]
## 13 [232, 430, 464, 948]
## 13 [1472, 325, 1594, 730]
## 14 [232, 432, 464, 948]
## 14 [1470, 327, 1600, 728]
## 14 [882, 57, 942, 156]
## 15 [232, 432, 464, 948]
## 15 [880, 55, 938, 156]
## 15 [1468, 323, 1600, 718]
## 16 [232, 430, 464, 948]
## 16 [882, 57, 938, 156]
## 16 [1466, 321, 1604, 712]
## 17 [232, 430,

## 152 [224, 434, 464, 948]
## 152 [1502, 408, 1662, 692]
## 153 [222, 436, 464, 948]
## 153 [1188, 226, 1322, 540]
## 153 [1502, 408, 1666, 692]
## 154 [218, 438, 464, 948]
## 154 [1194, 228, 1334, 530]
## 154 [1502, 408, 1666, 692]
## 155 [218, 440, 464, 948]
## 155 [1208, 228, 1338, 532]
## 156 [216, 442, 464, 948]
## 157 [214, 444, 464, 950]
## 157 [1218, 230, 1350, 534]
## 158 [214, 444, 462, 950]
## 158 [1234, 232, 1364, 532]
## 159 [212, 446, 460, 950]
## 159 [1242, 230, 1370, 534]
## 160 [212, 448, 460, 950]
## 160 [1248, 232, 1378, 532]
## 161 [212, 448, 454, 950]
## 161 [1252, 230, 1382, 534]
## 162 [214, 448, 450, 950]
## 162 [1264, 230, 1390, 536]
## 163 [214, 448, 452, 950]
## 163 [1270, 230, 1396, 538]
## 164 [216, 448, 458, 948]
## 164 [1280, 228, 1402, 543]
## 164 [1502, 414, 1662, 694]
## 165 [218, 448, 466, 950]
## 165 [1284, 228, 1404, 541]
## 165 [1502, 414, 1662, 692]
## 166 [222, 448, 478, 950]
## 166 [1290, 232, 1412, 541]
## 166 [1502, 412, 1664, 692]
## 167 [22

## 256 [188, 498, 418, 1064]
## 256 [1564, 281, 1698, 716]
## 256 [1312, 107, 1402, 385]
## 256 [1484, 232, 1592, 526]
## 257 [188, 496, 418, 1064]
## 257 [1564, 279, 1700, 718]
## 257 [1484, 240, 1592, 526]
## 257 [1314, 107, 1408, 387]
## 258 [190, 496, 420, 1064]
## 258 [1564, 279, 1706, 718]
## 258 [1482, 238, 1574, 536]
## 258 [1320, 109, 1414, 379]
## 259 [190, 494, 420, 1064]
## 259 [1566, 277, 1710, 720]
## 259 [1482, 236, 1576, 547]
## 260 [190, 494, 420, 1064]
## 260 [1570, 279, 1712, 720]
## 260 [1482, 240, 1580, 561]
## 261 [186, 494, 420, 1064]
## 261 [1572, 279, 1712, 720]
## 261 [1480, 244, 1580, 563]
## 262 [182, 494, 416, 1064]
## 262 [1572, 281, 1714, 718]
## 262 [1482, 248, 1586, 567]
## 263 [180, 496, 412, 1064]
## 263 [1572, 281, 1712, 718]
## 263 [1482, 248, 1584, 567]
## 263 [1334, 103, 1418, 385]
## 264 [186, 496, 406, 1062]
## 264 [1572, 281, 1712, 720]
## 264 [1482, 246, 1590, 571]
## 264 [1336, 103, 1420, 383]
## 265 [188, 496, 402, 1064]
## 265 [1572, 281, 1

## 363 [-2, 565, 426, 1078]
## 363 [1566, 277, 1704, 716]
## 363 [1390, 178, 1510, 547]
## 363 [1750, 762, 1926, 1078]
## 364 [-4, 567, 392, 1078]
## 364 [1568, 277, 1702, 716]
## 364 [1734, 736, 1926, 1074]
## 364 [1396, 186, 1506, 557]
## 364 [750, 71, 832, 188]
## 365 [-2, 569, 388, 1080]
## 365 [1568, 277, 1702, 716]
## 365 [1720, 647, 1924, 1078]
## 365 [1402, 188, 1506, 565]
## 365 [750, 71, 832, 188]
## 366 [0, 569, 352, 1081]
## 366 [1704, 589, 1928, 1080]
## 366 [1568, 275, 1702, 716]
## 366 [1404, 192, 1502, 565]
## 366 [210, 476, 428, 978]
## 367 [0, 571, 330, 1081]
## 367 [1702, 585, 1928, 1083]
## 367 [1404, 192, 1504, 567]
## 367 [1566, 275, 1702, 716]
## 367 [210, 476, 428, 978]
## 368 [1694, 577, 1926, 1081]
## 368 [-4, 571, 314, 1083]
## 368 [212, 476, 428, 978]
## 368 [1400, 192, 1504, 567]
## 368 [1566, 275, 1702, 716]
## 368 [748, 71, 832, 186]
## 369 [1694, 573, 1926, 1080]
## 369 [-2, 573, 304, 1080]
## 369 [212, 476, 428, 978]
## 369 [1396, 194, 1502, 567]
## 369

## 439 [-6, 583, 272, 1081]
## 439 [216, 482, 428, 980]
## 439 [1478, 343, 1648, 815]
## 440 [-4, 583, 272, 1081]
## 440 [214, 482, 428, 980]
## 440 [1472, 337, 1642, 815]
## 441 [-4, 587, 272, 1081]
## 441 [1462, 333, 1630, 815]
## 441 [214, 480, 428, 980]
## 441 [1568, 273, 1692, 714]
## 442 [-4, 591, 268, 1080]
## 442 [214, 480, 428, 980]
## 442 [1450, 333, 1616, 817]
## 442 [1558, 273, 1700, 716]
## 443 [-4, 591, 270, 1080]
## 443 [212, 480, 428, 980]
## 443 [1440, 333, 1608, 817]
## 443 [1550, 277, 1700, 714]
## 444 [-4, 589, 290, 1080]
## 444 [1422, 329, 1588, 808]
## 444 [214, 482, 428, 980]
## 444 [1550, 275, 1704, 714]
## 445 [-2, 587, 302, 1080]
## 445 [212, 482, 428, 980]
## 445 [1416, 325, 1582, 794]
## 445 [1558, 279, 1710, 714]
## 446 [0, 583, 336, 1081]
## 446 [1398, 315, 1566, 780]
## 446 [1554, 275, 1712, 716]
## 446 [210, 484, 428, 980]
## 447 [0, 579, 350, 1081]
## 447 [1394, 313, 1566, 764]
## 447 [1554, 275, 1712, 716]
## 447 [208, 484, 428, 980]
## 448 [6, 581, 36

...

## 696 [674, 186, 770, 500]
## 696 [1586, 293, 1736, 718]
## 696 [736, 150, 838, 432]
## 696 [1416, 166, 1502, 444]
## 696 [780, 81, 864, 262]
## 697 [674, 186, 770, 504]
## 697 [1586, 293, 1738, 722]
## 697 [738, 148, 838, 432]
## 697 [1414, 164, 1506, 442]
## 697 [1568, 77, 1656, 182]
## 698 [674, 188, 768, 506]
## 698 [1586, 295, 1740, 722]
## 698 [1410, 162, 1510, 438]
## 698 [738, 146, 838, 430]
## 698 [776, 87, 860, 216]
## 698 [1568, 77, 1646, 182]
## 699 [672, 188, 768, 510]
## 699 [1586, 293, 1740, 722]
## 699 [1406, 162, 1498, 438]
## 699 [736, 146, 838, 432]
## 699 [780, 89, 860, 216]
## 699 [1568, 77, 1644, 182]
## 700 [670, 190, 766, 512]
## 700 [1584, 295, 1740, 722]
## 700 [1404, 164, 1498, 442]
## 700 [734, 144, 838, 434]
## 700 [1570, 77, 1642, 184]
## 701 [668, 194, 766, 510]
## 701 [1582, 295, 1740, 722]
## 701 [1396, 164, 1496, 442]
## 701 [732, 144, 838, 436]
## 702 [666, 202, 768, 510]
## 702 [1578, 297, 1736, 722]
## 702 [1388, 164, 1496, 442]
## 702 [1570, 77, 1

## 798 [514, 210, 656, 549]
## 798 [1542, 303, 1710, 778]
## 798 [1210, 95, 1300, 305]
## 799 [514, 210, 656, 549]
## 799 [1210, 93, 1298, 303]
## 799 [1544, 303, 1712, 778]
## 800 [514, 208, 656, 549]
## 800 [1210, 93, 1298, 303]
## 800 [1544, 301, 1710, 776]
## 800 [1466, 289, 1564, 676]
## 801 [516, 206, 656, 549]
## 801 [1540, 299, 1708, 776]
## 801 [1210, 93, 1298, 299]
## 801 [1462, 289, 1564, 675]
## 802 [516, 206, 656, 549]
## 802 [1540, 297, 1710, 768]
## 802 [1208, 89, 1298, 289]
## 803 [516, 206, 656, 549]
## 803 [1548, 297, 1702, 762]
## 803 [1208, 89, 1298, 291]
## 803 [1448, 285, 1558, 661]
## 804 [518, 212, 656, 549]
## 804 [1550, 297, 1700, 750]
## 804 [1206, 89, 1296, 293]
## 804 [1446, 285, 1552, 651]
## 805 [518, 212, 656, 549]
## 805 [1546, 297, 1706, 738]
## 805 [1444, 283, 1554, 639]
## 805 [1206, 89, 1294, 293]
## 806 [518, 210, 656, 549]
## 806 [1546, 293, 1706, 734]
## 806 [1442, 281, 1552, 639]
## 807 [1546, 291, 1698, 734]
## 807 [518, 212, 656, 549]
## 807 [

## 902 [642, 174, 780, 480]
## 902 [542, 216, 684, 545]
## 902 [1374, 164, 1496, 474]
## 903 [644, 174, 782, 478]
## 903 [548, 216, 684, 545]
## 903 [1376, 164, 1494, 474]
## 903 [842, 77, 912, 184]
## 904 [646, 170, 782, 478]
## 904 [550, 216, 684, 545]
## 904 [1376, 164, 1492, 474]
## 904 [840, 77, 912, 226]
## 905 [650, 170, 784, 478]
## 905 [550, 214, 684, 543]
## 905 [1376, 162, 1498, 474]
## 905 [840, 77, 912, 240]
## 906 [656, 168, 784, 478]
## 906 [558, 214, 686, 543]
## 906 [1378, 162, 1496, 474]
## 906 [840, 79, 912, 232]
## 907 [656, 166, 784, 478]
## 907 [564, 214, 686, 541]
## 907 [1378, 160, 1500, 474]
## 907 [840, 77, 912, 238]
## 908 [570, 214, 684, 541]
## 908 [658, 164, 784, 476]
## 908 [1378, 160, 1504, 474]
## 909 [572, 212, 684, 545]
## 909 [1378, 160, 1504, 472]
## 909 [670, 164, 784, 468]
## 910 [576, 212, 682, 543]
## 910 [1380, 158, 1492, 466]
## 910 [1784, 178, 1880, 279]
## 911 [578, 212, 684, 541]
## 911 [1380, 158, 1496, 462]
## 911 [1794, 178, 1882, 281]
#

## 1006 [1370, 152, 1502, 446]
## 1006 [796, 95, 888, 331]
## 1006 [714, 127, 798, 365]
## 1007 [1370, 152, 1498, 444]
## 1007 [800, 97, 888, 327]
## 1007 [714, 123, 802, 367]
## 1008 [1370, 152, 1498, 444]
## 1008 [804, 95, 884, 325]
## 1008 [716, 125, 810, 361]
## 1009 [1372, 152, 1502, 446]
## 1009 [720, 123, 816, 361]
## 1009 [806, 95, 888, 319]
## 1010 [1372, 152, 1502, 446]
## 1010 [724, 123, 824, 361]
## 1010 [812, 95, 890, 309]
## 1011 [1372, 152, 1502, 446]
## 1011 [726, 123, 828, 361]
## 1012 [730, 125, 832, 361]
## 1012 [1370, 152, 1502, 446]
## 1012 [816, 93, 892, 264]
## 1013 [1370, 152, 1502, 446]
## 1013 [732, 125, 836, 361]
## 1013 [816, 93, 894, 264]
## 1014 [1370, 154, 1488, 444]
## 1014 [738, 125, 834, 359]
## 1014 [822, 95, 896, 262]
## 1015 [1370, 154, 1494, 444]
## 1015 [742, 123, 834, 359]
## 1015 [822, 93, 896, 260]
## 1016 [1370, 154, 1498, 446]
## 1016 [746, 119, 834, 359]
## 1016 [824, 93, 896, 254]
## 1017 [1368, 154, 1498, 446]
## 1017 [750, 119, 836, 351]


RuntimeError: No active exception to reraise

In [13]:
import time
from threading import Thread

def thread1():
    while 1:
        s = time.time()
        for i in range(200):
            a = 4
            a = pow(a, a)
            a = pow(a, a)
        e = time.time()
        print(e-s)
        time.sleep(0.0001)
# thread1()
# def thread1():
#     while 1:
#         a = 7
#         a = pow(a, a)
#         time.sleep(0.0001)

0.0003848075866699219
0.00039505958557128906
0.0003871917724609375
0.0003838539123535156
0.0003826618194580078
0.0003325939178466797
0.0003333091735839844
0.0003333091735839844
0.0003402233123779297
0.0003311634063720703
0.00033283233642578125
0.0003311634063720703
0.00033283233642578125
0.00033283233642578125
0.00033164024353027344
0.0003306865692138672
0.0003371238708496094
0.0003311634063720703
0.00033354759216308594
0.000331878662109375
0.0003311634063720703
0.00034689903259277344
0.0003368854522705078
0.0003364086151123047
0.00034236907958984375
0.00033664703369140625
0.00033664703369140625
0.0003364086151123047
0.0003361701965332031
0.0003368854522705078
0.0003364086151123047
0.000335693359375
0.0003426074981689453
0.00033664703369140625
0.0003361701965332031
0.00033664703369140625
0.0003364086151123047
0.00033664703369140625
0.0003364086151123047
0.0003361701965332031
0.00034356117248535156
0.0003364086151123047
0.0003364086151123047
0.0003361701965332031
0.0003361701965332031
0

0.00033354759216308594
0.00033354759216308594
0.0003268718719482422
0.00032639503479003906
0.0003333091735839844
0.00032639503479003906
0.00032639503479003906
0.00032639503479003906
0.00032711029052734375
0.00032591819763183594
0.0003268718719482422
0.00032639503479003906
0.0003314018249511719
0.00032591819763183594
0.00032639503479003906
0.0003261566162109375
0.0003268718719482422
0.0003261566162109375
0.0003266334533691406
0.0003266334533691406
0.0003273487091064453
0.00032639503479003906
0.0003268718719482422
0.0003261566162109375
0.0003266334533691406
0.0003261566162109375
0.0003268718719482422
0.0003261566162109375
0.0003268718719482422
0.00032639503479003906
0.0003268718719482422
0.0003261566162109375
0.00032639503479003906
0.00032591819763183594
0.0003261566162109375
0.0003261566162109375
0.00032639503479003906
0.0003523826599121094
0.00032639503479003906
0.0003325939178466797
0.00032591819763183594
0.0003256797790527344
0.00032639503479003906
0.00032591819763183594
0.0003263950

0.0003414154052734375
0.0003371238708496094
0.0003311634063720703
0.0003306865692138672
0.0003311634063720703
0.0003287792205810547
0.00033020973205566406
0.00032973289489746094
0.0003299713134765625
0.0003299713134765625
0.00033092498779296875
0.00032973289489746094
0.00032973289489746094
0.0003292560577392578
0.0003304481506347656
0.0003294944763183594
0.0003299713134765625
0.00032973289489746094
0.00032973289489746094
0.00033020973205566406
0.0003306865692138672
0.0003294944763183594
0.00032973289489746094
0.00033020973205566406
0.0003337860107421875
0.00032973289489746094
0.00033020973205566406
0.00032973289489746094
0.00032973289489746094
0.00032973289489746094
0.0003306865692138672
0.00032973289489746094
0.00033211708068847656
0.00032973289489746094
0.0003299713134765625
0.0003333091735839844
0.0003294944763183594
0.00032973289489746094
0.00033020973205566406
0.00032973289489746094
0.0003314018249511719
0.00032901763916015625
0.00032973289489746094
0.0003299713134765625
0.0003299

0.00033855438232421875
0.00033855438232421875
0.00033664703369140625
0.0003345012664794922
0.0003294944763183594
0.00032973289489746094
0.0003292560577392578
0.0003299713134765625
0.0003292560577392578
0.0003292560577392578
0.0003292560577392578
0.0003330707550048828
0.0003294944763183594
0.00033020973205566406
0.0003292560577392578
0.00032973289489746094
0.0003294944763183594
0.00032973289489746094
0.00033020973205566406
0.0003323554992675781
0.00033020973205566406
0.0003306865692138672
0.00033020973205566406
0.00033020973205566406
0.00033020973205566406
0.0003376007080078125
0.0003304481506347656
0.0003306865692138672
0.0003306865692138672
0.00033092498779296875
0.0003304481506347656
0.00033092498779296875
0.00033020973205566406
0.00033092498779296875
0.00033020973205566406
0.0003304481506347656
0.00033020973205566406
0.00033092498779296875
0.0003306865692138672
0.0003306865692138672
0.0003304481506347656
0.0003306865692138672
0.0003304481506347656
0.0003311634063720703
0.00033307075

0.0003509521484375
0.0003261566162109375
0.00033736228942871094
0.0003390312194824219
0.00033164024353027344
0.00033164024353027344
0.00033926963806152344
0.000331878662109375
0.00033164024353027344
0.0003323554992675781
0.00033164024353027344
0.000331878662109375
0.0003314018249511719
0.0003314018249511719
0.0003371238708496094
0.00033211708068847656
0.000331878662109375
0.0003380775451660156
0.00033211708068847656
0.0003323554992675781
0.0003314018249511719
0.00033211708068847656
0.0003364086151123047
0.0003311634063720703
0.00033164024353027344
0.000331878662109375
0.0003311634063720703
0.00033164024353027344
0.00033211708068847656
0.00033164024353027344
0.00033664703369140625
0.000331878662109375
0.000331878662109375
0.00033164024353027344
0.0003311634063720703
0.00033164024353027344
0.000331878662109375
0.000331878662109375
0.00033664703369140625
0.0003314018249511719
0.0003311634063720703
0.000331878662109375
0.0003314018249511719
0.0003314018249511719
0.000331878662109375
0.0003

...

0.0002751350402832031
0.0002765655517578125
0.0002701282501220703
0.0002694129943847656
0.0002701282501220703
0.0002694129943847656
0.00026988983154296875
0.0002696514129638672
0.0002696514129638672
0.0002694129943847656
0.00026988983154296875
0.0002734661102294922
0.0002701282501220703
0.0002696514129638672
0.0002696514129638672
0.0002694129943847656
0.0002701282501220703
0.0002696514129638672
0.0002701282501220703
0.0002696514129638672
0.00027441978454589844
0.00026917457580566406
0.00026917457580566406
0.0002696514129638672
0.00026988983154296875
0.00026988983154296875
0.0002701282501220703
0.0002751350402832031
0.0002701282501220703
0.00026988983154296875
0.0002696514129638672
0.0002694129943847656
0.0002696514129638672
0.0002694129943847656
0.0002701282501220703
0.0002694129943847656
0.0002701282501220703
0.0002701282501220703
0.0002694129943847656
0.00027179718017578125
0.0002703666687011719
0.00026917457580566406
0.00026988983154296875
0.0002696514129638672
0.0002694129943847656

0.00028061866760253906
0.00028061866760253906
0.0002734661102294922
0.0002739429473876953
0.000274658203125
0.00027680397033691406
0.0002732276916503906
0.00027298927307128906
0.00027298927307128906
0.00027251243591308594
0.00027298927307128906
0.00027251243591308594
0.00027370452880859375
0.00027298927307128906
0.00027370452880859375
0.0002732276916503906
0.00027298927307128906
0.00027251243591308594
0.0002732276916503906
0.0002727508544921875
0.0002732276916503906
0.00027298927307128906
0.0002734661102294922
0.0002732276916503906
0.0002760887145996094
0.00027298927307128906
0.0002732276916503906
0.0002732276916503906
0.0002732276916503906
0.0002732276916503906
0.0002734661102294922
0.00027298927307128906
0.0002734661102294922
0.000274658203125
0.0002739429473876953
0.00027298927307128906
0.0002732276916503906
0.0002727508544921875
0.00027298927307128906
0.0002727508544921875
0.0002734661102294922
0.0002727508544921875
0.0002734661102294922
0.0002732276916503906
0.0002734661102294922


0.0002758502960205078
0.00027823448181152344
0.0002703666687011719
0.0002696514129638672
0.0002701282501220703
0.00026988983154296875
0.00026988983154296875
0.0002694129943847656
0.0002696514129638672
0.0002694129943847656
0.00026988983154296875
0.00026988983154296875
0.00026988983154296875
0.00026988983154296875
0.0002701282501220703
0.0002696514129638672
0.0002701282501220703
0.0002694129943847656
0.0002696514129638672
0.00027441978454589844
0.000270843505859375
0.0002703666687011719
0.00027060508728027344
0.00027060508728027344
0.00027108192443847656
0.0002703666687011719
0.00027108192443847656
0.0002703666687011719
0.0002734661102294922
0.000270843505859375
0.000270843505859375
0.0002703666687011719
0.00027108192443847656
0.0002701282501220703
0.00027108192443847656
0.00027060508728027344
0.000270843505859375
0.000270843505859375
0.000270843505859375
0.00027060508728027344
0.000270843505859375
0.00027060508728027344
0.000270843505859375
0.00027060508728027344
0.00027108192443847656

0.00028204917907714844
0.00027298927307128906
0.0002739429473876953
0.0002830028533935547
0.0002803802490234375
0.0002751350402832031
0.0002751350402832031
0.00027561187744140625
0.0002753734588623047
0.0002753734588623047
0.00027561187744140625
0.00027561187744140625
0.0002741813659667969
0.00027489662170410156
0.0002751350402832031
0.0002751350402832031
0.0002758502960205078
0.0002753734588623047
0.0002753734588623047
0.0002753734588623047
0.00027561187744140625
0.0002753734588623047
0.00027561187744140625
0.0002753734588623047
0.0002753734588623047
0.0002810955047607422
0.0002753734588623047
0.00027561187744140625
0.0002753734588623047
0.00027441978454589844
0.0002741813659667969
0.0002741813659667969
0.0002765655517578125
0.0002791881561279297
0.00028014183044433594
0.0002741813659667969
0.0002739429473876953
0.0002741813659667969
0.000274658203125
0.00027489662170410156
0.000274658203125
0.0002765655517578125
0.0002751350402832031
0.00027489662170410156
0.0002751350402832031
0.000

KeyboardInterrupt: 

In [ ]:
import time
from threading import Thread

def thread1():
    while 1:
        s = time.time()
        for i in range(200):
            a = 4
            a = pow(a, a)
            a = pow(a, a)
        e = time.time()
        print(e-s)
        time.sleep(0.0001)
def run(frame_list):
    for i in range(10) : 
        start = time.time()
        for input_data in frame_list : 
            x, scale_list = pe.run_inference([input_data],[])
th1=Thread(target=thread1)
th2=Thread(target=thread1)
th3=Thread(target=thread1)
th4=Thread(target=thread1)
th5=Thread(target=run, kwargs={'frame_list':frame_list})
th1.start()
th2.start()
th3.start()
th4.start()
th5.start()

In [22]:
import cv2
import torch
# from interface_dynamic import PEOPLE_DETECTOR
from interface_people_hj import PEOPLE_DETECTOR
import time
import numpy as np
from threading import Thread
torch.cuda.init()

model_queue
def cpu_computation():
    size=10000
    arr = np.ones([size,size,3])
    arr += 1
    result = arr * arr
    print(result)
        
                
weights = '/DATA_17/ij/peopleNet_test/best_model_people.trt'

pe = PEOPLE_DETECTOR()
pe.load(weights)

def analysis_thread():
    for i in range(10) : 
        start = time.time()
        for input_data in frame_list : 
            x, scale_list = pe.run_inference([input_data],[])

        print(f'end : {time.time() -start}')
        
# Thread(target=analysis_thread)

NameError: name 'model_queue' is not defined

In [18]:
th1.start()

In [19]:
th2.start()

In [20]:
th3.start()

In [21]:
th4.start()

In [5]:
import numpy as np
from collections import deque

def decoder():
    for i in range(100000):
        a = 0.124684413
        a **= a
        arr += 1
        result = arr * arr
        print(result.shape)
        
# cpu_computation()

(10000, 10000, 3)
(10000, 10000, 3)
(10000, 10000, 3)
(10000, 10000, 3)
(10000, 10000, 3)
(10000, 10000, 3)
(10000, 10000, 3)
(10000, 10000, 3)
(10000, 10000, 3)
(10000, 10000, 3)
(10000, 10000, 3)
(10000, 10000, 3)


KeyboardInterrupt: 

# PeopleNet with Stream

In [ ]:
import cv2
import torch
# from interface_dynamic import PEOPLE_DETECTOR
from interface_people_hj_stream import PEOPLE_DETECTOR
import time
torch.cuda.init()


                
weights = '/DATA_17/ij/peopleNet_test/best_model_people.trt'

pe = PEOPLE_DETECTOR()
pe.load(weights)

print('frame load done.')
for i in range(10) : 
    start = time.time()
    for input_data in frame_list : 
        x, scale_list = pe.run_inference([input_data],[])

    print(f'end : {time.time() -start}')
raise
from collections import deque
import time
from threading import Thread
import traceback

model_queue = deque()
inf_queue = deque()
route_queue = deque()
end_queue = deque()

model_batch = 1
count = 0

def decoder(frame_list):
    for i in range(10):
        for frame in frame_list:
            model_queue.append(frame)
            
def get_queue():
    while True:
        try:
            if not model_queue:
                time.sleep(0.0001)
                continue
            input_batch = []
            
            iter_times = len(model_queue)
            for _ in range(iter_times):
                if len(input_batch) >= model_batch:
                    break
                data = model_queue.popleft()
                inf_queue.append(data)
                
        except Exception as e:
            print(e)
            print(traceback.format_exc())
        finally:
            pass
        
def inference():
    while True:
        try:
            if not inf_queue:
                time.sleep(0.0001)
                continue
            # 큐에서 가져오기
            input_data = inf_queue.popleft()
            
            # 추론
            x, scale_list = pe.run_inference([input_data],[])
            
            # 라우팅 스텝 큐로 전달
            route_queue.append(x)
                
        except Exception as e:
            print(e)
            print(traceback.format_exc())
        finally:
            pass
        
def route(count):
    flag = False
    history = 0.0
    while True:
        try:
            if not route_queue:
                time.sleep(0.0001)
                continue
            if not flag:
                s = time.time()
                flag = True
            input_data = route_queue.popleft()
            
#             x, scale_list = pe.run_inference([input_data],[])
            
            count += 1
            if count % 1198 == 0:
                print(f'{count}, {time.time()-s-history}')
                history = time.time()-s
#             route_queue.append(x)
                
        except Exception as e:
            print(e)
            print(traceback.format_exc())
        finally:
            pass

th1 = Thread(target=get_queue)
th2 = Thread(target=inference)
th3 = Thread(target=route, kwargs={'count':count})

decoder(frame_list)
th1.start()
th2.start()
th3.start()

In [1]:
from interface_people_hj_stream import PEOPLE_DETECTOR
from collections import deque
import time
from threading import Thread
import torch
import torch.cuda as cuda
import cv2
import os

torch.cuda.init()


def proc(video_path):
    frame_list = []
    cap = cv2.VideoCapture(video_path)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)  # float
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float
    fps = cap.get(cv2.CAP_PROP_FPS)
    cnt = 0
    if cap.isOpened() :    
        while True:
            cnt+=1
            ret, frame = cap.read()
            if ret == False :
                break
            if cnt < 0:
                continue
#             if cnt > 300:
#                 break
            infe_frame = torch.from_numpy(frame).to(torch.device("cuda"),non_blocking=True)
            infe_frame = infe_frame.permute(2, 0, 1) 
            input_data = dict()
            input_data["framedata"] = {"frame":infe_frame}
            input_data["framedata"]['meta'] = {'source' : {'channel_id' : str(1_10), 'frame_count' : cnt }}
            input_data["bbox"] = [0,0,infe_frame.shape[2],infe_frame.shape[1]]
            input_data["scenario"] = "s"   
            input_data["data"] = None   
            frame_list.append(input_data)
    return frame_list
video_path = '/DATA_17/ij/peopleNet_test/0001_compressed.mp4'
frame_list = proc(video_path)
print('frema load done : ', len(frame_list))

frema load done :  1198


In [3]:
def run_stream(idx, pe, stream, sts, frame_list, result_pool, model_queue):
    
    st1 = cuda.Stream(device='cuda')
    
    while 1:
        if not model_queue[idx]:
            time.sleep(0.001)
            continue
        with stream(st1):
            frame_data = model_queue[idx].popleft()
            res = pe.run_infer([frame_data], [], idx)
            print(time.time())
#             time.sleep(0.0001)
        

weights = '/DATA_17/ij/peopleNet_test/best_model_people.trt'

pe = PEOPLE_DETECTOR()
pe.load(weights)

number_of_context = 4
stream = cuda.stream
# sts = dict()
# for i in range(10):
#     sts[i] = cuda.Stream(device="cuda")
model_queue = [deque() for _ in range(number_of_context)]
sts = None
result_pool = {}
th_list = []
for idx in range(number_of_context):
    th = Thread(target=run_stream, kwargs={'idx':idx,'pe':pe, 'stream':stream, 'sts':sts, 
                                           'frame_list':frame_list, 'result_pool':result_pool, 'model_queue':model_queue})
    th_list.append(th)

for th in th_list :
    th.start()
    
for i in range(3):
    t1 = time.time()
    print(i, 'times')
    for idx, frame_data in enumerate(frame_list):
        stream_idx = idx % number_of_context
        model_queue[stream_idx].append(frame_data)
        time.sleep(0.0001)
    t2 = time.time()
    print('put queue time : ', t2-t1)
    time.sleep(10)
    
                   

[TensorRT] WARNING: Using an engine plan file across different models of devices is not recommended and is likely to affect performance or even cause errors.
[TensorRT] WARNING: TensorRT was linked against cuDNN 8.2.1 but loaded cuDNN 8.2.0
[TensorRT] WARNING: TensorRT was linked against cuDNN 8.2.1 but loaded cuDNN 8.2.0
[TensorRT] WARNING: TensorRT was linked against cuDNN 8.2.1 but loaded cuDNN 8.2.0
[TensorRT] WARNING: TensorRT was linked against cuDNN 8.2.1 but loaded cuDNN 8.2.0
[TensorRT] WARNING: TensorRT was linked against cuDNN 8.2.1 but loaded cuDNN 8.2.0


0 set_binding
1 set_binding
2 set_binding
3 set_binding
0 times
1675130723.8643894
1675130723.8671548
1675130723.86805
1675130723.8698711
1675130723.8724506
1675130723.8809485
1675130723.885515
1675130723.8880398
1675130723.8890622
1675130723.889552
1675130723.8967004
1675130723.9010582
1675130723.901599
1675130723.9028108
1675130723.9035318
1675130723.9107072
1675130723.9114819
1675130723.9128354
1675130723.915386
1675130723.9204226
1675130723.9205937
1675130723.9213889
1675130723.9261646
1675130723.931033
1675130723.9313114
1675130723.931526
1675130723.9353967
1675130723.940994
1675130723.941637
1675130723.9453173
1675130723.9460976
1675130723.9526129
1675130723.9545338
1675130723.9576895
1675130723.9619715
1675130723.963531
1675130723.9639778
1675130723.9710069
1675130723.9724813
1675130723.9733427
1675130723.9805682
1675130723.980748
1675130723.9842892
1675130723.9881268
1675130723.9910917
1675130723.9980712
1675130723.999207
1675130724.0004268
1675130724.0010104
1675130724.007318


1675130725.2456443
1675130725.2506287
1675130725.2557526
1675130725.2565253
1675130725.2582114
1675130725.264406
1675130725.2656043
1675130725.2659729
1675130725.266373
1675130725.2738624
1675130725.2764485
1675130725.279901
1675130725.2803714
1675130725.2818623
1675130725.2852194
1675130725.2909582
1675130725.2911603
1675130725.2925065
1675130725.2932956
1675130725.3022873
1675130725.3030558
1675130725.3062336
1675130725.3106434
1675130725.3118377
1675130725.3122067
1675130725.3175895
1675130725.321006
1675130725.3214355
1675130725.326471
1675130725.3298488
1675130725.3345702
1675130725.3350716
1675130725.336287
1675130725.3425746
1675130725.343655
1675130725.345773
1675130725.3464797
1675130725.3540869
1675130725.355343
1675130725.3555255
1675130725.3608284
1675130725.3621237
1675130725.367098
1675130725.367486
1675130725.3720617
1675130725.3728592
1675130725.3775501
1675130725.378236
1675130725.3839967
1675130725.3841486
1675130725.3850079
1675130725.3921626
1675130725.3954127
16751

1675130726.6710672
1675130726.6753297
1675130726.6760828
1675130726.6808162
1675130726.6838174
1675130726.6875606
1675130726.6878576
1675130726.691266
1675130726.6952057
1675130726.6984618
1675130726.7003639
1675130726.7005906
1675130726.7061772
1675130726.7094805
1675130726.7124438
1675130726.7127385
1675130726.7161715
1675130726.7186391
1675130726.7238207
1675130726.7241516
1675130726.7247849
1675130726.731766
1675130726.7341056
1675130726.737373
1675130726.7379627
1675130726.7400177
1675130726.742988
1675130726.748081
1675130726.750594
1675130726.7561085
1675130726.7580013
1675130726.758236
1675130726.7610936
1675130726.7671902
1675130726.767851
1675130726.7703266
1675130726.7723956
1675130726.77762
1675130726.7783537
1675130726.78374
1675130726.7871509
1675130726.7878754
1675130726.7939665
1675130726.7958622
1675130726.796058
1675130726.8016803
1675130726.8030035
1675130726.8095179
1675130726.8098
1675130726.8117979
1675130726.8124166
1675130726.8190181
1675130726.8192484
167513072

1675130734.7321892
1675130734.73761
1675130734.7386081
1675130734.7389572
1675130734.7449827
1675130734.7487755
1675130734.7491093
1675130734.7502775
1675130734.7541032
1675130734.7601576
1675130734.7606049
1675130734.7610655
1675130734.7658157
1675130734.7721195
1675130734.7733407
1675130734.773554
1675130734.7738483
1675130734.7811904
1675130734.786119
1675130734.7868197
1675130734.7870305
1675130734.7929068
1675130734.7934794
1675130734.7997317
1675130734.8031096
1675130734.807874
1675130734.8101847
1675130734.8139756
1675130734.8146653
1675130734.8176444
1675130734.8213649
1675130734.8234632
1675130734.8256407
1675130734.8258898
1675130734.8316996
1675130734.8321793
1675130734.8372495
1675130734.8399637
1675130734.8433764
1675130734.8469858
1675130734.8502572
1675130734.8517957
1675130734.8584614
1675130734.8637478
1675130734.8646321
1675130734.8716133
1675130734.8732347
1675130734.873781
1675130734.874284
1675130734.8826594
1675130734.8839605
1675130734.884239
1675130734.8847415
1

1675130736.1671886
1675130736.1705956
1675130736.173716
1675130736.1761312
1675130736.1803756
1675130736.1806178
1675130736.1810052
1675130736.1864793
1675130736.1909518
1675130736.1932933
1675130736.1935177
1675130736.1987383
1675130736.1998985
1675130736.2035146
1675130736.2065258
1675130736.2082796
1675130736.2088153
1675130736.2155185
1675130736.2160127
1675130736.2163353
1675130736.2216976
1675130736.2258635
1675130736.226572
1675130736.2275968
1675130736.2330728
1675130736.2376075
1675130736.2379196
1675130736.238283
1675130736.2431612
1675130736.2484279
1675130736.2491298
1675130736.2495627
1675130736.2556875
1675130736.2571511
1675130736.2610173
1675130736.2655892
1675130736.2660794
1675130736.268428
1675130736.2716777
1675130736.2782333
1675130736.278438
1675130736.2804384
1675130736.2813406
1675130736.2883787
1675130736.2887251
1675130736.2922564
1675130736.2929368
1675130736.2976286
1675130736.2978425
1675130736.3032253
1675130736.3037732
1675130736.308072
1675130736.3083196

1675130737.606741675130737.606835

1675130737.6139524
1675130737.6148868
1675130737.6154537
1675130737.6189682
1675130737.6219091
1675130737.6277537
1675130737.6336727
1675130737.6344085
1675130737.6383886
1675130737.640242
1675130737.6407294
1675130737.646944
1675130737.64739
1675130737.650059
1675130737.6559126
1675130737.6568537
1675130737.6569881
1675130737.657371
1675130737.6670244
1675130737.6680105
1675130737.6685476
1675130737.6687336
1675130737.6751165
1675130737.68059
1675130737.680794
1675130737.6844144
1675130737.6884155
1675130737.6896055
1675130737.689786
1675130737.6950924
1675130737.6986723
1675130737.7009907
1675130737.7056968
1675130737.7059555
1675130737.7120948
1675130737.712714
1675130737.7129228
1675130737.714048
1675130737.7182245
1675130737.723582
1675130737.7244427
1675130737.7303977
1675130737.7313876
1675130737.735903
1675130737.7409985
1675130737.7421696
1675130737.7448485
1675130737.748992
1675130737.7530806
1675130737.7571573
1675130737.7611158
1675130737.

1675130745.9431543
1675130745.9502404
1675130745.9554534
1675130745.955859
1675130745.9574692
1675130745.9599156
1675130745.9651096
1675130745.9711225
1675130745.9730856
1675130745.973509
1675130745.9741452
1675130745.979538
1675130745.9842496
1675130745.9845161
1675130745.984724
1675130745.9894922
1675130745.9947522
1675130745.9953022
1675130745.9955153
1675130746.0006154
1675130746.0025523
1675130746.0079994
1675130746.008267
1675130746.0101683
1675130746.0124857
1675130746.0180225
1675130746.018316
1675130746.0207021
1675130746.0236585
1675130746.029232
1675130746.0298376
1675130746.0302355
1675130746.0352736
1675130746.0407972
1675130746.0410585
1675130746.041292
1675130746.046155
1675130746.0521626
1675130746.0524414
1675130746.0527618
1675130746.056242
1675130746.062388
1675130746.0669687
1675130746.0672011
1675130746.0680573
1675130746.0746076
1675130746.0748594
1675130746.0803444
1675130746.0809147
1675130746.0854895
1675130746.0857
1675130746.0915415
1675130746.0918446
1675130

1675130747.3813894
1675130747.3819666
1675130747.3858747
1675130747.3899968
1675130747.3907034
1675130747.3934102
1675130747.3969123
1675130747.4026275
1675130747.403173
1675130747.403468
1675130747.407263
1675130747.4109507
1675130747.41716
1675130747.417688
1675130747.4215539
1675130747.4218132
1675130747.4274728
1675130747.429274
1675130747.4324758
1675130747.434925
1675130747.4376197
1675130747.4428258
1675130747.4451046
1675130747.4467335
1675130747.450561
1675130747.4525144
1675130747.4531956
1675130747.459086
1675130747.4593546
1675130747.4640772
1675130747.4644122
1675130747.470082
1675130747.4703612
1675130747.4727483
1675130747.4756184
1675130747.4814086
1675130747.4817207
1675130747.4824603
1675130747.4860353
1675130747.492073
1675130747.4946733
1675130747.4949899
1675130747.4978795
1675130747.50365
1675130747.5049906
1675130747.5066445
1675130747.5102553
1675130747.5136223
1675130747.5173097
1675130747.5193412
1675130747.519544
1675130747.5251803
1675130747.5254505
16751307

In [4]:
# 스트림 생성
1675130727.4928489-1675130723.8643894

3.6284594535827637

In [13]:
# 스트림 생성
1675129567.9063108-1675129563.0070016

4.899309158325195

In [5]:
# 컨텍스트 3개
1675129288.7913437-1675129285.142454

3.6488897800445557

# YoloXs Load

In [11]:

# from interface_dynamic import PEOPLE_DETECTOR
from interface_yoloxs import DETECTOR
import time
torch.cuda.init()


                
weights = '/DATA_17/media_test/model_manager/engines/yoloxs_int8/yoloxs_best.trt'

pe = DETECTOR()
pe.load(weights)
for i in range(1) : 
    start = time.time()
    for input_data in frame_list : 
        x, scale_list = pe.run_inference(input_data,[])

    print(f'end : {time.time() -start}')
raise


KeyboardInterrupt: 

# YoloXs + Stream

In [2]:
# YoloXs + Stream
# from interface_dynamic import PEOPLE_DETECTOR
from interface_yoloxs_stream import DETECTOR
import time
import torch
torch.cuda.init()


                
weights = '/DATA_17/media_test/model_manager/engines/yoloxs_int8/yoloxs_best.trt'

pe = DETECTOR()
pe.load(weights)


# for i in range(10) : 
#     start = time.time()
#     for input_data in frame_list : 
#         x, scale_list = pe.run_inference(input_data,[])

#     print(f'end : {time.time() -start}')
# raise


[TensorRT] WARNING: TensorRT was linked against cuDNN 8.2.1 but loaded cuDNN 8.2.0
[TensorRT] WARNING: TensorRT was linked against cuDNN 8.2.1 but loaded cuDNN 8.2.0


In [3]:
import time
from threading import Thread
IS_DONE = {'is_end':False}
def thread1(IS_DONE):    
    while 1:
        if IS_DONE['is_end'] == True:
            break
        s = time.time()
        for i in range(62):
            a = 4
            a = pow(a, a)
            a = pow(a, a)
        e = time.time()
#         print(e-s)
        time.sleep(0.0001)
    print("thread done")
    
def run(frame_list):
    for input_data in frame_list : 
        x, scale_list = pe.run_inference(input_data,[])
        time.sleep(0.0001)

    IS_DONE['is_end'] = True
    print("run done")    
for i in range(1):  
    IS_DONE['is_end'] = False
    t1 = time.time()        
    th1=Thread(target=thread1, kwargs={'IS_DONE':IS_DONE})
    th4=Thread(target=thread1, kwargs={'IS_DONE':IS_DONE})
    th5=Thread(target=run, kwargs={'frame_list':frame_list})
    th1.start()
    th4.start()
    th5.start()
    th5.join()
    th1.join()
    th4.join()
    t2 = time.time()
    print(t2-t1)



run done
thread done
thread done
9.349658012390137
run done
thread done
thread done
6.9045562744140625
run done
thread done
thread done
8.017923593521118
run done
thread done
thread done
7.905444860458374
run done
thread done
thread done
7.735955715179443
run done
thread done
thread done
7.347796440124512
run done
thread done
thread done
8.131444454193115
run done
thread done
thread done
7.359301805496216
run done
thread done
thread done
6.7219202518463135
run done
thread done
thread done
7.5839197635650635


In [3]:
from interface_yoloxs import DETECTOR
import time
import torch
import torch.multiprocessing as mp
from collections import deque


torch.cuda.init()
weights = '/DATA_17/media_test/model_manager/engines/yoloxs_int8/yoloxs_best.trt'


# dt = DETECTOR()
# dt.load(weights)

pi_proc = dt.parse_input
pre_proc = dt.preprocess
inference = dt.inference
post_proc = dt.postprocess
po_proc = dt.parse_output

def pre(pre_proc, infer_queue, frame_list):
    for frame_batch in frame_list:
        pi_result = pi_proc(frame_batch)
        pre_result, scale_list = pre_proc(pi_result)
        pre_result.share_memory_()
        infer_queue.put(pre_result)
#         infer_queue.append(pre_result)
        time.sleep(0.0001)
        
def inf(inference, infer_queue):
    while 1:
        if not infer_queue:
            time.sleep(0.0001)
            continue

        input_batch = infer_queue.get()
#         input_batch = infer_queue.popleft()

        infer_result = inference(input_batch)
        print(infer_result)
        
mp.set_start_method("spawn")
manager = mp.Manager()
infer_queue = manager.Queue()
# infer_queue = deque()
# post_queue = manager.Queue()
pre_p = mp.Process(target=pre, args=(pre_proc, infer_queue, frame_list,))
inf_p = mp.Process(target=inf, args=(inference, infer_queue,))
pre_p.start()
inf_p.start()
inf_p.join()
pre_p.join()
# pre(pre_proc, infer_queue, frame_list)
# inf(inference, infer_queue)

RuntimeError: context has already been set

In [ ]:
def inf(inference, infer_queue)
    while 1:
        if not infer_queue:
            time.sleep(0.0001)
            continue

        input_batch = infer_queue.popleft()

        infer_result = inference(input_batch)
        print(infer_result)

In [8]:
frame_batch = frame_list[0]
print(len(frame_batch))
pi_result = pi_proc(frame_batch)
pre_result = pre_proc(pi_result)
print(pre_result)

16
(tensor([[[[ 52.,  53.,  55.,  ...,  47.,  45.,  45.],
          [ 50.,  52.,  56.,  ...,  44.,  47.,  51.],
          [ 48.,  52.,  56.,  ...,  47.,  46.,  46.],
          ...,
          [144., 144., 144.,  ..., 144., 144., 144.],
          [144., 144., 144.,  ..., 144., 144., 144.],
          [144., 144., 144.,  ..., 144., 144., 144.]],

         [[ 44.,  45.,  47.,  ...,  44.,  42.,  42.],
          [ 42.,  44.,  48.,  ...,  41.,  44.,  48.],
          [ 40.,  44.,  48.,  ...,  44.,  43.,  43.],
          ...,
          [144., 144., 144.,  ..., 144., 144., 144.],
          [144., 144., 144.,  ..., 144., 144., 144.],
          [144., 144., 144.,  ..., 144., 144., 144.]],

         [[ 42.,  43.,  45.,  ...,  41.,  39.,  39.],
          [ 40.,  42.,  46.,  ...,  38.,  41.,  45.],
          [ 38.,  42.,  46.,  ...,  41.,  40.,  40.],
          ...,
          [144., 144., 144.,  ..., 144., 144., 144.],
          [144., 144., 144.,  ..., 144., 144., 144.],
          [144., 144., 144., 

In [ ]:
from collections import deque
import time
from threading import Thread
import traceback

# model_queue = deque()
# inf_queue = deque()
# route_queue = deque()
# end_queue = deque()


def decoder(frame_list, test):
    for i in range(10):
        for frame in frame_list:
            test.model_queue.append(frame)
class TEST:
    def __init__(self):
        self.model_queue = deque()
        self.inf_queue = deque()
        self.route_queue = deque()
        self.end_queue = deque()
        
        self.count = 0
        self.model_batch = 1
        
        
    def load(self, pe):
        self.pe = pe
        
        
        
    def get_queue(self):
        while True:
            try:
                if not self.model_queue:
                    time.sleep(0.0001)
                    continue
                input_batch = []

                iter_times = len(self.model_queue)
                for _ in range(iter_times):
                    if len(input_batch) >= self.model_batch:
                        break
                    data = self.model_queue.popleft()
                    self.inf_queue.append(data)

            except Exception as e:
                print(e)
                print(traceback.format_exc())
            finally:
                pass

    def inference(self):
        while True:
            try:
                if not self.inf_queue:
                    time.sleep(0.0001)
                    continue
                # 큐에서 가져오기
                input_data = self.inf_queue.popleft()

                # 추론
                x, scale_list = self.pe.run_inference([input_data],[])

                # 라우팅 스텝 큐로 전달
                self.route_queue.append(x)

            except Exception as e:
                print(e)
                print(traceback.format_exc())
            finally:
                pass

    def route(self):
        flag = False
        history = 0.0
        while True:
            try:
                if not self.route_queue:
                    time.sleep(0.0001)
                    continue
                if not flag:
                    s = time.time()
                    flag = True
                input_data = self.route_queue.popleft()

    #             x, scale_list = pe.run_inference([input_data],[])

                self.count += 1
                if self.count % 1198 == 0:
                    print(f'{self.count}, {time.time()-s-history}')
                    history = time.time()-s
    #             route_queue.append(x)

            except Exception as e:
                print(e)
                print(traceback.format_exc())
            finally:
                pass

In [ ]:

from interface_people_hj_stream import PEOPLE_DETECTOR
import time
torch.cuda.init()


                
weights = '/DATA_17/ij/peopleNet_test/best_model_people.trt'

pe = PEOPLE_DETECTOR()
pe.load(weights)

test = TEST()
test.load(pe)
th1 = Thread(target=test.get_queue)
th2 = Thread(target=test.inference)
th3 = Thread(target=test.route)


In [ ]:
decoder(frame_list, test)
th1.start()
th2.start()
th3.start()
# th1.join()
# th2.join()
# th3.join()

In [ ]:
print(len(frame_list))

In [ ]:
print(len(model_queue))
print(len(inf_queue))
print(len(route_queue))
print(count)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms


model = models.resnet50()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

dataset = datasets.FakeData(
    size=1000,
    transform=transforms.ToTensor())
loader = DataLoader(
    dataset,
    num_workers=1,
    pin_memory=True
)

model.to('cuda')

for data, target in loader:
    data = data.to('cuda', non_blocking=True)
    target = target.to('cuda', non_blocking=True)
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    
print('Done')

In [33]:
with open('./people_stream.log', 'r') as f:
    stream_lines = f.readlines()
    
# print(len(stream_lines))

with open('people_cvt.log', 'w') as f:
    for line in stream_lines:
        if '##' not in line:
            continue
        sp_lines = line.replace('*', ('\n'))
        f.write(sp_lines)
#         print(sp_lines)
#         for sl in sp_lines:
#             if sl == '\n':
#                 continue
#             else:
#                 f.write(sl)

In [ ]:
with open('./people_ori.log', 'r') as f:
    ori_lines = f.readlines()
    
with open('./people_stream.log', 'r') as f:
    stream_lines = f.readlines()

In [37]:
with open('./people_cvt.log', 'r') as f:
    ori_lines = f.readlines()
    
with open('./people_gt.log', 'r') as f:
    stream_lines = f.readlines()

In [38]:
print(len(ori_lines))
print(len(stream_lines))

5288
4920


In [39]:
ori_dict = {}
stream_dict = {}
for ori in ori_lines:
    if 'not label' not in ori:
        print(ori)
        fn, bbox = ori.split('## ')[-1].split(' [')
        if fn not in ori_dict:
            ori_dict[fn] = bbox
            
    
for stream in stream_lines:
    if 'not label' not in stream:
#         print(stream)
        fn, bbox = stream.split('## ')[-1].split(' [')
        if fn not in stream_dict:

            stream_dict[fn] = bbox

## 1 [0, 0, 429, 996]

## 1 [681, 0, 2310, 639]

## 2 [351, 684, 228, 528]

## 2 [351, 690, 231, 534]

## 2 [351, 687, 228, 531]

## 2 [348, 693, 228, 534]

## 2 [348, 687, 225, 525]

## 2 [354, 687, 222, 525]

## 2 [423, 621, 192, 471]

## 2 [423, 618, 204, 471]

## 2 [1584, 543, 132, 396]

## 2 [423, 615, 201, 453]

## 2 [1584, 546, 135, 393]

## 2 [1584, 543, 135, 390]

## 2 [426, 624, 201, 465]

## 2 [1584, 549, 135, 387]

## 2 [351, 681, 225, 534]

## 2 [1584, 549, 135, 396]

## 2 [1584, 549, 132, 390]

## 2 [423, 621, 201, 465]

## 2 [348, 693, 225, 537]

## 2 [423, 624, 192, 471]

## 2 [1584, 546, 135, 390]

## 2 [441, 615, 168, 459]

## 2 [426, 615, 189, 468]

## 2 [1584, 549, 138, 390]

## 2 [1560, 354, 96, 192]

## 2 [1560, 354, 96, 195]

## 2 [1584, 546, 135, 384]

## 2 [1563, 354, 96, 192]

## 2 [1560, 351, 102, 195]

## 2 [354, 693, 228, 528]

## 3 [351, 687, 228, 531]

## 3 [351, 687, 231, 540]

## 3 [351, 684, 228, 531]

## 3 [348, 693, 228, 546]

## 3 [351, 687, 222, 53

In [41]:
ori_count = 0
str_count = 0
for i in range(1, 1199):
    key = str(i)
    if key not in ori_dict.keys() and key not in stream_dict.keys():
        print(key, 'anywhere key')
        continue
    
    if key not in ori_dict.keys():
        print(i, 'str', stream_dict[key])
        str_count += 1
        
    
    if key not in stream_dict.keys():
        print(i, 'ori', ori_dict[key])
        ori_count += 1
        
    if key in stream_dict.keys() and key in ori_dict.keys():
        if len(stream_dict[key]) != len(ori_dict[key]):
            print(len(stream_dict[key]), len(ori_dict[key]))
        
#     print(ori_dict[str(i)], stream_dict[str(i)])
print(ori_count , str_count)

20 16
0 0


In [86]:
import time
for i in range(10):
    s = time.time()
    for i in range(60):
        a = 4
        a = pow(a, a)
        a = pow(a, a)
    e = time.time()
    print(e-s)

0.00014138221740722656
0.00013518333435058594
0.00013399124145507812
0.0014960765838623047
0.00013780593872070312
0.00013494491577148438
0.00013494491577148438
0.00013518333435058594
0.00013494491577148438
0.0001347064971923828


In [19]:
from torch.multiprocessing import Queue

In [9]:
a = Queue(maxsize=100)

3.029362916946411

In [10]:
1674097160.2532315-1674097151.5783832

8.674848318099976

In [18]:
import time

post_batch = 16
s = time.time()
scale_list = [0.33333 for _ in range(post_batch)]
e = time.time()
print(e-s)

0.00012874603271484375


In [11]:
1674104116.3375227-1674104113.4644065

2.8731162548065186

In [3]:
1674110693.348405-1674110691.4110234

1.9373815059661865

In [5]:
1674110796.119363-1674110791.1776445

4.941718578338623

In [7]:
4.941718578338623-1.009-1.078-1.433-0.497-0.093-0.1-0.123 0.002~ 0.003


0.7317185783386233

In [8]:
1674110744.048673-1674110741.03874

3.0099329948425293

In [10]:
0.12184147262573242*25

3.04603681564331

In [11]:
1674178319.5274713-1674178319.3776803

0.14979100227355957

In [12]:
1674178318.4362628-1674178316.191408

2.2448549270629883

In [15]:
1674178605.653896-1674178601.412843

4.241053104400635

In [16]:
1674178553.3125904-1674178553.7317824

-0.4191920757293701

In [17]:
1674178992.1146321-1674178987.7536705

4.360961675643921

In [18]:
import torch
torch.__version__

'1.10.1+cu113'

In [1]:
1674616357.4534917-1674616353.4901152

3.963376522064209

In [46]:
1674637347.6386802-1674637343.2525573

4.386122941970825

In [84]:
len(lines)

74

In [57]:
with open('./result.txt', 'r') as f:
    lines = f.readlines()
print(len(lines))
time_count = {
    'return' : 0,
    'infer' : 0,
    'put' : 0,
    'get' : 0,
}
# time_count = {
#     'return' : [],
#     'infer' : [],
#     'put' : [],
#     'get' : [],
# }

key_dict = {
    0 : 'get',
    1 : 'return',
    2 : 'infer',
    3 : 'end',
    4 : 'put'
}
time_counts = 0
for line in lines:
    if 'end_time' in line:
        continue
#     if 'start_time' in sp:
#         continue
    if '##' in line:
        continue
    sp = line.split('//')
#     print(sp)
    time_counts += 1
    for i, spp in enumerate(sp):
        
        spp =spp.replace(' ', '').replace('\n','')
        time = float(spp.split(':')[-1])
        
        if i == 3:
            continue
#         print(i, time)
        key = key_dict[i]
        time_count[key]+=time
print(time_count)
print(time_counts)
# total_time = sum([time for time in time_count.values()])
print(time_count['return'] + time_count['infer'])

74
{'return': 3.6951425075531006, 'infer': 0.3766207695007324, 'put': 0.001781463623046875, 'get': 3.667933464050293}
74
4.071763277053833


In [52]:
#### test time
1674637086.551693-1674637085.5457082

1.0059847831726074

In [6]:
1675328773.4305372-1675328772.0723987

1.3581385612487793

In [11]:

print('infer start to end', 1675330431.2541976-1675330430.2702644)

print('post start to end',1675330432.661037-1675330431.3754768)

infer start to end 0.9839332103729248
post start to end 1.285560131072998


In [9]:
0.2+0.09+0.27+0.9+0.1+0.1+1.34+0.4+0.18

3.58

In [1]:
import torch
import torch.multiprocessing as mp

In [2]:
in_, out_ = mp.Pipe()

In [4]:
# A = torch.zeros([100,100,3]).to(torch.device('cuda'))
A = torch.zeros([100,100,3])

In [5]:
out_.send(A)

In [6]:
result = in_.recv()

In [7]:
print(result)

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        ...,

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])


In [52]:
def test():
    di = [1,2,3,4,5]
    yield from di
            
def test2():
    for ctx in test():
        yield ctx

In [53]:
test()

<generator object test at 0x7f315c1da2d0>

In [54]:
a = [1,2,3]
b = iter(a)


In [59]:
next(b)

StopIteration: 

In [60]:
with open('untitled.txt', 'r') as f:
    lines = f.readlines()
# with open('new_text.txt', 'w') as f1:
    
#     for line in lines:
#         new_line = line.replace('\n', ' ')
#         f1.write(new_line)

In [63]:
number_list = []
for line in lines:
    numbers = line.split(': ')

#     print(numbers)
    for number in numbers:
        number_list.append(number.split(', ')[-1])
#     number_list.append(number)
    
# for i in range(300):
#     if i not in number_list:
#         print(i)

In [64]:
for i in range(1200):
    if str(i) not in number_list:
        print(i)

0
1
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
1199


In [10]:
1675228320.3819835-1675228320.2777321

0.10425138473510742

In [9]:
1675228320.8645322-1675228309.998634

10.865898132324219

In [8]:
1675227640.778756-1675227631.7720215

9.006734371185303

In [17]:
import multiprocessing as mp
from multiprocessing import managers

In [23]:
syncm = managers.SyncManager()

In [24]:
queue = syncm.Queue()

AssertionError: server not yet started

In [3]:
import torch

In [5]:
A = torch.ones([3,10,10]).to(torch.device('cuda'))

In [27]:
A._typed_storage()

AttributeError: 'Tensor' object has no attribute '_typed_storage'

NameError: name 'zfill' is not defined

In [12]:
shm = mp.shared_memory(create=True, size=A.nbytes)


AttributeError: module 'multiprocessing' has no attribute 'shared_memory'

In [5]:
import torch
print(torch)

<module 'torch' from '/opt/conda/lib/python3.7/site-packages/torch/__init__.py'>


In [8]:
B = torch.tensor(A)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [9]:
B

tensor([[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.,

In [18]:
print(q._send)

AttributeError: 'function' object has no attribute '_send'

In [25]:
1675231002.2409964-1675230989.6311567

12.609839677810669

In [19]:
0.19+0.083+0.18+0.085+1.04+1.04+0.005+0.184+0.185+0.187+0.172+0.32+0.94+0.088

4.699000000000001